
# Using inspect module to browse through badly documented modules

Yet another among old notebooks I had that would be a shame not to preserve even though today this is even easier to do than back then.

We want to grab another module, and print out all existing classes, their methods and all functions present in that module. Added to that we want to be able to get source code for each method and function and we want to make that simpler to use, meaning we want to generate summaries of the entire code to speed up learning new codebase.

Inspect module, that comes in standard with Python, has some very interesting functions that'll make our job here straight out easy. First off let's run through the inspect module functions fast:

```
>>> help(inspect.getmembers)
Help on function getmembers in module inspect:

getmembers(object, predicate=None) Return all members of an object as (name, value) pairs sorted by name. Optionally, only return members that satisfy a given predicate.
```

Predicates are given by any one of the `inspect.isclass`, `isfunction`, `ismethod,` `iscode` functions of inspect class.
Later on we'll also use:

```
>>> help(inspect.indentsize)
Help on function indentsize in module inspect:

indentsize(line) Return the indent size, in spaces, at the start of a line of text.</code>
```

Those are pretty much all functions from inspect module we'll be using. One to get members of objects, and 5 others to clasify what kind of members they are. We're going to sort the retrieved objects into a class, and write functions to make a nice print out of those lists, and to retrieve the source code.

In [1]:
import inspect

class InspectIt():
    """Class that wraps a lot of `inspect` modules functionality to generate summaries
    of poorly documented modules.
    
    Parameters
    ----------
    module : module
        module, library or some object we want to inspect
    """
    def __init__(self, module):
        self.obj = module
        self.classes = inspect.getmembers(module, predicate=inspect.isclass)
        self.methods ={owner[0]:inspect.getmembers(owner[1], predicate=inspect.isfunction)
                       for owner in self.classes #run through all classes in module
                       }
        self.functions = inspect.getmembers(module, predicate=inspect.isfunction)
        self.snippets = inspect.getmembers(module, predicate=inspect.iscode)


    def _print_block(self, start, read):
        """Prints tabulated blocks of text to fit more text legibly on the screen. """
        #inspect module provides indentation count of the first line 
        n_tabs = inspect.indentsize(read[start])

        # firstlineno gets set after declaration
        # to include it in print go back 1 line 
        in_block=True
        print(read[start-1], end="")
        while in_block:             
            for char in range(n_tabs):
                try:
                    # if starting line is doc_string, print it
                    if read[start][n_tabs:n_tabs+3] == '"""':
                        while not read[start].endswith('"""\n'):
                            print(read[start], end="")
                            start+=1
                        start+=1
                    # if the line starts with anything else
                    # we've escaped the code block and it's time to stop printing
                    if read[start][char] != " ":
                        in_block=False
                    else:
                        print(read[start], end="")
                        start+=1
                        
                except:
                    pass
        

    def _get_func_code(self, name):
        """Retrieves the source code of a function of a given name."""
        file = open(self.obj.__file__)
        read = file.readlines()
        func = getattr(self.obj, str(name)).__code__
        func_line = func.co_firstlineno
        self._print_block(func_line,read)
        
            
    def _get_method_code(self, owner, name):
        """Retrieves the source code of a method of a given name."""
        print("Method {} of clas {}\n".format(name, owner))
        file = open(self.obj.__file__)
        read = file.readlines()
        clas = getattr(self.obj, owner)
        func = getattr(clas, str(name)).__code__
        func_line = func.co_firstlineno
        self._print_block(func_line, read)
        print("\n\n")


    def get_code(self, name, owner_class=False):
        """Given a name returns the source code of a function, or if class is
        also specified returns the source code of the method of that class. 
        """
        found = False
        # First we check if the sent name is found in list of functions 
        # and print the source code if it is
        for func in self.functions:
            if name == func[0]:
                self._get_func_code(name)
                found = True
        # otherwise suspect it's a method name and see if additional
        # information was provided
        if owner_class:
            try:
                self._get_method_code(owner_class, name)
                found = True
            except:
                return("No methods of functions found")
        else:
            # if owner name has  not been sent we check if a method can be found 
            # in the list of all methods.
            for owner, methods in self.methods.items():
                for method_name, method in methods:
                    if method_name == name:
                        self._get_method_code(owner, method_name)
                        found = True

        if not found:
            raise NameError("No methods or functions with that name found")

    def _print_class_methods(self):
        """Prints all class method names in neat tabulated format."""
        for clas in self.classes:
            print("Class:", end="\t")
            print(clas[0])
            for method1, method2 in zip(self.methods[clas[0]][0::2],self.methods[clas[0]][1::2]) :
                print("\t\t{0:^25s}  ||  {1:^25s}".format(method1[0],method2[0]))
                #if your function has more than 25 characters you're doing
                #something wrong
        
    def _print_func(self):
        """Prints all function names in neat tabulated format."""
        print("Functions:")
        for func1, func2 in zip(self.functions[0::2],self.functions[1::2]) :
            print("\t{0:<25s}  ||  {1:<25s}".format(func1[0],func2[0]))
            #if your function has more than 25 characters you're doing
            #something wrong

    @property
    def summary(self):
        """Generates a summary of all functions names, class names and methods
        in a neat tabulated format.
        """
        self._print_class_methods()
        print()
        self._print_func()

There is a distinct problem that happens while inspecting everything non-module in form. Prime examples are packages, that are not called in their init.py file. You will not be able to retrieve any kind of usefull information from those. F.e. `mpl_toolkits`, or any packages that define `load()` wherein they import their code in module like manner.

You will be able to retrieve only the load function but to inspect any particular module from that package, you will have to import it individually by providing a direct path to its source files. This heavily defeats the purpose of this code. This is solvable by walking the mro but too much hasle to do.

There is also the problem with builtins. Often in more advanced programs builtin functions are rewritten with additional or completely changed implementations which can be ingnored by `InspectIt` class.

The last issue to mention would most likely be inheritance related problems. Child classes that do not re-implement inherited methods and functions don't get screened when making a summary and therefore it's possible to see functions that aren't defined in that particular class to be used while looking at source-code.

Lastly, for demo and fun lets inspect `inspect` module.

In [2]:
import inspect


inspected = InspectIt(inspect)
print("    SUMMARY")
print("-------------------------------------------------------------------------------------------------")
inspected.summary

print("\n\n    SOURCE CODE OF GETCOMMENTS FUNCTION")
print("-------------------------------------------------------------------------------------------------")
inspected.get_code("getcomments")

print("\n\n    SOURCE CODE OF INIT METHOD OF BLOCKFINDER CLASS")
print("-------------------------------------------------------------------------------------------------")
inspected.get_code("__init__", owner_class="BlockFinder")

    SUMMARY
-------------------------------------------------------------------------------------------------
Class:	ArgInfo
		     __getnewargs__        ||           __new__         
		        __repr__           ||           _asdict         
Class:	ArgSpec
		     __getnewargs__        ||           __new__         
		        __repr__           ||           _asdict         
Class:	Arguments
		     __getnewargs__        ||           __new__         
		        __repr__           ||           _asdict         
Class:	Attribute
		     __getnewargs__        ||           __new__         
		        __repr__           ||           _asdict         
Class:	BlockFinder
		        __init__           ||         tokeneater        
Class:	BoundArguments
		         __eq__            ||        __getstate__       
		        __init__           ||          __repr__         
		      __setstate__         ||       apply_defaults      
Class:	ClosureVars
		     __getnewargs__        ||           __new__         